<a href="https://colab.research.google.com/github/HammadN98/pytorch/blob/main/10_pytorch_intro_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducao ao Pytorch 2

In [1]:
import torch
print(torch.__version__)

2.5.1+cu121


## Pytorch 1.X

In [2]:
import torch
import torchvision

model = torchvision.models.vgg16()

## Com a chegada do Pytorch 2.X

In [3]:
import torch
import torchvision

model_vgg = torchvision.models.vgg16() #QUase todos os modelos

compiled_model = torch.compile(model_vgg)

### Treino


###Teste

## 0. Configurando o ambiente

In [4]:
## Versao recente ja

## 1. Informacoes da GPU

PQ pegar info da GPU?
Pq o Pytorch 2 funciona melhor nas novas GPUs NVIDIA, foi criado para isso

* Ter uma GPU com score de +8.0 o bagulho fica doido

In [5]:
!nvidia-smi

Tue Dec 10 14:19:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 1.1 Configurando o dispositivo de maneira global

Antigamente tinha que configurar o dispositivo dos tensor/modelos:

* `tensor.to(device)`
* `model.to(device)`

Porem com o pytorch 2.X e possivel configurar com "Context manager" e tambem de maneira global.


In [6]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

#Configurando com "context manager"
with torch.device(device):
  layer = torch.nn.Linear(20, 30)
  print(f"Pessos da camada estao no dispositivo: {layer.weight.device}")
  print(f"Camada criada no dispositivo: {layer(torch.randn(128, 20)).device}")


Pessos da camada estao no dispositivo: cuda:0
Camada criada no dispositivo: cuda:0


In [7]:
import torch


#COnfigurando o disopositivo de maneira global
torch.set_default_device(device)

#Todos os tensors e objetos Pytorch criados apartir daqui estaram no dispositivo configurado sem a necessidade de usar o .to()
layer = torch.nn.Linear(20, 30)
print(f"Pessos da camada estao no dispositivo: {layer.weight.device}")
print(f"Camada criada no dispositivo: {layer(torch.randn(128, 20)).device}")


Pessos da camada estao no dispositivo: cuda:0
Camada criada no dispositivo: cuda:0


## 2. Configurnado experimentos

Para testar a "nova" velocidade do pytorch, serao realizados 4 experimentos, sendo eles com:
 * Modelo: ResNet50
 * Dados: CIFAR10 do torchvision
 * Epochs: 5 e 3x5(Com calculo da media)
 * Batch size 128
 * Iamge size 224

In [8]:
import torch
import torchvision

print(f"Versao do Pytorch: {torch.__version__}")
print(f"Versao do Torchvision: {torchvision.__version__}")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando: {device}")


Versao do Pytorch: 2.5.1+cu121
Versao do Torchvision: 0.20.1+cu121
Usando: cuda


### 2.1 Criando o modelo e os transforms

* ResNet50

In [9]:
model_weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2
transforms = model_weights.transforms()

transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [10]:
model = torchvision.models.resnet50(weights=model_weights)
model

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 144MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
#Contando a quantidade de parametros
total_params = sum(
    param.numel() for param in model.parameters() #COntando todos os paraemtros
    #param.numel() for param in model.parameters() if param.requires_grad = True ||Contando apenas os parametros treinaveis.
)
total_params

25557032

**NOTA**  Os speedsup do 2.x serao mais notaveius quando for possivel usar a maior quantidade de processaento do GPU. Oq siginifica que modelo maiores(com mais paramentros treinaveis) levaram um tempo maior para serem treinados mas serao relativamente mais rapidos que modelos menores. EX: 1m de parametros leva 10 min, 25M de parametrso leva 25 min

In [12]:
def create_model(num_classes=10):
  """
  Cria um modelo ResNet50 com seus transformadores e devolve os dois
  """
  model_weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2
  transforms = model_weights.transforms()
  model = torchvision.models.resnet50(weights=model_weights)

  #Ajustando a camada head para o nosso problema
  model.fc = torch.nn.Linear(in_features=2048,
                             out_features=num_classes)

  return model, transforms

model, transforms = create_model()
transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

### 2.2 "Speedups" sao mais notaveis quando um grande porcao da GPU esta sendo usada

Considerando que as GPUs modernas sao *Rapidas* performando operacoes, e possivel notar o aumentos de velocidade relativo quando se usa o maximo de dados possiveis na GPU.

Na pratica, se quer usar o maximop da memoria da GPU

* Aumentando o valor do batch size (USamos de 32, porem vale se possivel aumentar 128, 256, 512
* Aumentando o tamanhodoas imagens
* Aumentando o tamanho do modelo
* Diminuindo a transferencia de dados ( CPU x GPU)

Assim o tempo relativo deve melhorar, claro que o tempo tatal tambem ira aumentar


### 2.3 Chacado os limites de memoria da GPU disponivel

Usando ```torch.cuda ```



In [15]:
total_free_gpu_memory, total_gpu_memory = torch.cuda.mem_get_info()
print(f"Memoria livre: {round(total_free_gpu_memory * 1e-9, 3)}GB")
print(f"Memoria total da GPU: {round(total_gpu_memory * 1e-9, 3)}GB")


Memoria livre: 15.453GB
Memoria total da GPU: 15.836GB


Se a memoria for > 16gb usar batch de 128, caso contrario batch de 32

In [17]:
total_free_gpu_memory_gb = round(total_gpu_memory * 1e-9, 3)
if total_free_gpu_memory_gb >= 16:
  BATCH_SIZE = 128
  IMAGE_SIZE = 224
  print(f"memoria disponivel: {total_free_gpu_memory_gb} GB, usando batch_size de {BATCH_SIZE} e image_size de {IMAGE_SIZE}")
else:
  BATCH_SIZE = 32
  IMAGE_SIZE = 128
  print(f"memoria disponivel: {total_free_gpu_memory_gb} GB, usando batch_size de {BATCH_SIZE} e image_size de {IMAGE_SIZE}")

memoria disponivel: 15.836 GB, usando batch_size de 32 e image_size de 128


In [18]:
transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [19]:
transforms.crop_size = 224
transforms.resize_size = 128
print(f"Transforms atualizados: \n{transforms}")

Transforms atualizados: 
ImageClassification(
    crop_size=224
    resize_size=128
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


### 2.4 Mais speedups com TF32

TensorFloat32 -> um tipo de dado que combina Float32 e Float16:
* Range com a quantidade de bits do FP32
* Precisao com com a quantidade de bits do FP16

In [21]:
torch.backends.cuda.matmul.allow_tf32 = True